## Importing necessary libraries

In [ ]:
import os
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.utils import plot_model
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

## Spliting the data folder into : Train and Test 

In [ ]:
# Set the path to your data directory
data_dir = "C:/Users/DELL/Desktop/datatest/D-E"

# Get the list of all folders names
image_files = os.listdir(data_dir)
print("All folders names:", image_files)

In [ ]:
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

## Define the model architecture

In [ ]:
# Define the input image size and number of classes
img_width = 1280
img_height = 720
batch_size = 32
num_classes = 2

In [ ]:
# input layer
inputs = Input(shape=(224, 224, 3))

# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# Flatten
x = Flatten()(x)

# Fully connected layers
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer
outputs = Dense(num_classes, activation='softmax')(x)


In [ ]:
# Create the model
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Define the data generators for image augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255, # rescales the pixel values to the range [0, 1]
    rotation_range=20, # randomly rotates the images by up to 20 degrees
    width_shift_range=0.1, # randomly shifts the images horizontally by up to 10% of the image width
    height_shift_range=0.1, # randomly shifts the images vertically by up to 10% of the image height
    brightness_range=(0.8, 1.2), # randomly adjusts the brightness of the images by a factor between 0.8 and 1.2
    shear_range=0.2, # randomly applies shearing transformations to the images
    zoom_range=0.2, # randomly applies zooming transformations to the images
    horizontal_flip=True, # randomly flips the images horizontally
    fill_mode='nearest' #  fills in any empty pixels resulting from the transformations using the nearest neighboring pixel value
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load the training and testing data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)

In [ ]:
# Define early stopping callback
early_stop = EarlyStopping(monitor=['val_loss', 'val_accuracy'], patience=10)

## Model training

In [ ]:
# Train the model
history = model.fit(
    train_data, 
    epochs=1000, 
    steps_per_epoch=train_data.n // train_data.batch_size,
    validation_data=test_data,
    validation_steps=test_data.n // test_data.batch_size,
    callbacks=[early_stop]
)

## Plot the training and validation loss and accuracy over the epochs

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Confusion matrix

In [ ]:
# Predict the classes for the validation data
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true classes for the validation data
y_true = test_data.classes

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot the confusion matrix using seaborn
sns.heatmap(cm, annot=True, cmap='Blues')